# CCS potential with LAMMPS  for ZnO

In this example, we will run CCS potentials with LAMMPS for multicomponent systems. ZnO  CCS potential is used for this illustration. Note that the exponential head part in **CCS_params.json** is ignored when LAMMPS spline tables are created. The exponential part is avoided because LAMMPS performs a pre-interpolation on the tabular data, and often pre-interpolation done with exponential head gives worse results. In general, a higher resolution should be used while generating spline table for LAMMPS. By default, the gridsize used for CCS is divided by 10.  

In [ ]:
# Generates CCS.table file readable by LAMMPS
from ccs.scripts.jsonTotable import asecalcTotable
tags=asecalcTotable("CCS_params.json",scale=10)  # Controls the resolution of the gridsize; gridsize=dr/scale.
tags

In [ ]:
!head CCS.table

In [ ]:
import json
import copy
import matplotlib.pyplot as plt

from ase import Atoms
from ase.calculators.lammpsrun import LAMMPS
from ccs.ase_calculator.ccs_ase_calculator import CCS
from ase.io import read,write
from ase.io.trajectory import Trajectory
from ase.optimize import BFGS
from sklearn.metrics import mean_squared_error


with open ('CCS_params.json','r') as f:
    CCS_params=json.load(f)


# Parameters for LAMMPS 
parameters = {
              'pair_style': 'table spline 1000',   # The number of elements chosen taken for pre-interpolation
              'pair_coeff': ['1 1  CCS.table O-O 5.9999990640749274', # Specify Rcut for each pair
                             '1 2  CCS.table O-Zn 5.9999991089416',
                             '2 2  CCS.table Zn-Zn 5.999995708965134'],
#              'pair_write': ['1 2 500 r  1.2999991089415996 5.9999991089416 table.txt table']
}



# Single point evaluation

In [ ]:
struct=read('POSCAR')
lammps = LAMMPS(parameters=parameters,keep_tmp_files=False)
ccs= CCS(CCS_params)


struct_ase=copy.deepcopy(struct)

struct.calc = lammps
print("Energy from LAMMPS  pre optmization: ", struct.get_potential_energy())
struct_ase.calc=ccs
print("Energy from CCS calculator pre optmization:", struct_ase.get_potential_energy())

print ("MSE on energy between ase calc and LAMMPS: {}".format((struct_ase.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between ase calc and LAMMPS: {}".format(mean_squared_error(struct.get_forces(),struct_ase.get_forces())))




# Optimization

In [ ]:
traj = Trajectory('LAMMPS.traj', 'w', struct)
dyn=BFGS(struct)
dyn.attach(traj)
dyn.run(fmax=0.05)
print("Energy from LAMMPS after optimization: ", struct.get_potential_energy())


traj = Trajectory('CCS.traj', 'w', struct_ase)
dyn=BFGS(struct_ase)
dyn.attach(traj)
dyn.run(fmax=0.05)
print("Energy from CCS calculator after optimization:", struct_ase.get_potential_energy())

print ("MSE on energy between ase calc and LAMMPS: {}".format((struct_ase.get_potential_energy()-struct.get_potential_energy())**2))
print ("MSE on forces between ase calc and LAMMPS: {}".format(mean_squared_error(struct.get_forces(),struct_ase.get_forces())))



**Note** : 
* Onebody energy contributons in CCS_params file was set to 0 for easy comparison